### A. Schema (modelo lógico)

Define:
- Columnas
- Tipos de datos
- Nullabilidad

Importancia:
- Permite optimización (Catalyst)
- Reduce errores de inferencia
- Mejora performance

Idea clave:
👉 El schema **no afecta el paralelismo**, pero sí el **plan de ejecución**


# 1. Configuracion base

In [35]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType
from pyspark.sql.functions import col
spark = SparkSession.builder \
    .appName("spark_labs") \
    .master("local[6]") \
    .config("spark.executor.cores", "2") \
    .config("spark.executor.memory", "4g") \
    .config("spark.default.parallelism", "12") \
    .config("spark.sql.shuffle.partitions", "12") \
    .config("spark.sql.adaptive.enabled", "true") \
    .getOrCreate()

print(spark.sparkContext.uiWebUrl)

# 1. Componentes:

### A. Schema
Define:
- Columnas
- Tipos de datos
- Nullabilidad

Importancia:
- Permite optimización (Catalyst)
- Reduce errores de inferencia
- Mejora performance

Idea clave:
👉 El schema **no afecta el paralelismo**, pero sí el **plan de ejecución**


In [36]:
# NO REALIZAR DE ESTA FORMA
df_bad = spark.read.csv("./src/ventas.csv", header=True)
print("-"*10)
df_bad.printSchema() # Ver el schema
print("-"*10)
df_bad.rdd.getNumPartitions() # Ver cuántas particiones tiene el DataFrame

----------
root
 |-- id_venta: string (nullable = true)
 |-- fecha: string (nullable = true)
 |-- monto: string (nullable = true)
 |-- pais: string (nullable = true)
 |-- cliente_id: string (nullable = true)
 |-- producto: string (nullable = true)

----------


1

In [37]:
# FORMA CORRECTA
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType, DateType

schema = StructType([
    StructField("id_venta", IntegerType(), False),
    StructField("fecha", DateType(), True),
    StructField("monto", DoubleType(), True),
    StructField("pais", StringType(), True)
])

df = spark.read.csv(
    "./src/ventas.csv",
    header=True,
    schema=schema
)
print("-"*10)
df.printSchema() # Ver el schema
print("-"*10)

----------
root
 |-- id_venta: integer (nullable = true)
 |-- fecha: date (nullable = true)
 |-- monto: double (nullable = true)
 |-- pais: string (nullable = true)

----------


### B. Particiones (modelo físico)

Las particiones definen:
- Cómo se **divide** el DataFrame
- Qué tareas se ejecutan en paralelo

Ejemplo mental:
- 6 cores → máximo 6 tasks simultáneas
- 2 particiones → solo 2 cores trabajan

Regla:
👉 El paralelismo depende de las particiones


repartition(6)

* Siempre redistribuye los datos
* Siempre hace shuffle
* Crea 6 particiones balanceadas
* Es costoso, pero correcto para paralelismo

In [38]:
df.rdd.getNumPartitions() # Ver cuántas particiones tiene el DataFrame

1

In [39]:
df_repart = df.repartition(6)
df_repart.rdd.getNumPartitions()

6

coalesce(6)

* NO redistribuye los datos
* NO hace shuffle
* Solo junta particiones existentes
* Solo tiene sentido para reducir particiones

In [40]:
df_small = df_repart.coalesce(3)
df_small.rdd.getNumPartitions()

3

### C. Linaje (DAG)

El linaje es:
- El grafo de transformaciones
- NO es ejecución inmediata

Ejemplo:
```text
read → filter → select → join → groupBy

In [41]:
df_filtrado = df.filter(df.monto > 100)
df_select = df_filtrado.select("pais", "monto")
df_group = df_select.groupBy("pais").sum("monto")

df_group.show()   # ← ACCIÓN


+----+----------+
|pais|sum(monto)|
+----+----------+
|  CL|     850.0|
|  PE|    1770.5|
|  MX|    2800.0|
|  CO|     700.0|
+----+----------+



In [42]:
df_group.explain(True)


== Parsed Logical Plan ==
'Aggregate ['pais], ['pais, unresolvedalias('sum(monto#243))]
+- Project [pais#244, monto#243]
   +- Filter (monto#243 > cast(100 as double))
      +- Relation [id_venta#241,fecha#242,monto#243,pais#244] csv

== Analyzed Logical Plan ==
pais: string, sum(monto): double
Aggregate [pais#244], [pais#244, sum(monto#243) AS sum(monto)#261]
+- Project [pais#244, monto#243]
   +- Filter (monto#243 > cast(100 as double))
      +- Relation [id_venta#241,fecha#242,monto#243,pais#244] csv

== Optimized Logical Plan ==
Aggregate [pais#244], [pais#244, sum(monto#243) AS sum(monto)#261]
+- Project [pais#244, monto#243]
   +- Filter (isnotnull(monto#243) AND (monto#243 > 100.0))
      +- Relation [id_venta#241,fecha#242,monto#243,pais#244] csv

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[pais#244], functions=[sum(monto#243)], output=[pais#244, sum(monto)#261])
   +- Exchange hashpartitioning(pais#244, 12), ENSURE_REQUIREMENTS, [plan_id=406]

In [43]:
df_group.explain("formatted")


== Physical Plan ==
AdaptiveSparkPlan (6)
+- HashAggregate (5)
   +- Exchange (4)
      +- HashAggregate (3)
         +- Filter (2)
            +- Scan csv  (1)


(1) Scan csv 
Output [2]: [monto#243, pais#244]
Batched: false
Location: InMemoryFileIndex [file:/C:/Users/User_Kenny/Documents/_KENNEDY_MOLINA_PROYECTOS/pyspark-advanced-databricks/01_data_modeling/src/ventas.csv]
PushedFilters: [IsNotNull(monto), GreaterThan(monto,100.0)]
ReadSchema: struct<monto:double,pais:string>

(2) Filter
Input [2]: [monto#243, pais#244]
Condition : (isnotnull(monto#243) AND (monto#243 > 100.0))

(3) HashAggregate
Input [2]: [monto#243, pais#244]
Keys [1]: [pais#244]
Functions [1]: [partial_sum(monto#243)]
Aggregate Attributes [1]: [sum#264]
Results [2]: [pais#244, sum#265]

(4) Exchange
Input [2]: [pais#244, sum#265]
Arguments: hashpartitioning(pais#244, 12), ENSURE_REQUIREMENTS, [plan_id=406]

(5) HashAggregate
Input [2]: [pais#244, sum#265]
Keys [1]: [pais#244]
Functions [1]: [sum(monto#243)]
Aggre